In [ ]:
pip install selenium webdriver-manager requests bs4


In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Runs in background, remove if you want to see browser
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Open the target website
website_url = "https://courses.genzcareer.org/complete-interview-prepration-id23242555de32f5/"
driver.get(website_url)
time.sleep(5)  # Wait for the page to load

# Expand all dropdowns
dropdowns = driver.find_elements(By.CLASS_NAME, "accordion-toggle")  # Update if needed
for dropdown in dropdowns:
    try:
        driver.execute_script("arguments[0].scrollIntoView();", dropdown)
        ActionChains(driver).move_to_element(dropdown).click().perform()
        time.sleep(2)  # Wait for content to load
    except:
        pass

# Extract all PDF links
pdf_links = []
links = driver.find_elements(By.TAG_NAME, "a")

for link in links:
    href = link.get_attribute("href")
    if href and href.endswith(".pdf"):
        pdf_links.append(href)

# Close the browser
driver.quit()

# Create a folder to save PDFs
save_folder = "downloaded_pdfs"
os.makedirs(save_folder, exist_ok=True)

# Download all PDFs
for pdf_link in pdf_links:
    pdf_name = os.path.join(save_folder, pdf_link.split("/")[-1])
    print(f"Downloading: {pdf_name}")
    
    with requests.get(pdf_link, stream=True) as r:
        with open(pdf_name, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

print("Download complete!")
